In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu

# Read recipe inputs
jokes_tokenized_by_text_joined = dataiku.Dataset("jokes_tokenized_by_text_joined")
jokes_tokenized_by_text_joined_df = jokes_tokenized_by_text_joined.get_dataframe()

test = dataiku.Dataset("test")
test_df = test.get_dataframe()

my_jokes=dataiku.Dataset("jokes_copy_to_join_prepared")
my_jokes_df=my_jokes.get_dataframe()


jokes_info_df=jokes_tokenized_by_text_joined_df.copy()
rating_df=test_df.copy()
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

In [2]:
jokes_info_df.head()

dfm  count mot  user  moyenne_user  moyenneBlague
0    1      3  00     0          6.26       1.687813
1    1      3  00     4          8.11       1.687813
2    1      3  00     8         -1.60       1.687813
3    1      3  00    11         -0.29       1.687813
4    1      3  00    16         -2.28       1.687813

In [3]:
my_jokes_df.head()

id                                               text
0  95  ["just","thought","critic","someon","walk","mi...
1   1  ["man","visit","doctor","doctor","say","bad","...
2  83  ["woman","say","place","mess","c","mon","need"...
3  17  ["mani","men","take","screw","light","bulb","o...
4  40  ["mani","irishmen","take","chang","lightbulb",...

In [4]:
rating_df.head()

user  blague  rating
0     0      20    1.17
1     0       7   -0.78
2     0      16   -0.19
3     0      44   -5.87
4     0      18    2.96

In [5]:
rating_df[(rating_df["user"]==0)   ]

user  blague  rating
0      0      20    1.17
1      0       7   -0.78
2      0      16   -0.19
3      0      44   -5.87
4      0      18    2.96
5      0      29   -2.23
6      0      36    1.26
7      0      15   -1.31
8      0      19   -0.29
9      0      31    0.15
10     0      48   -8.20
11     0       8    6.89
12     0      67    2.23
13     0      43   -7.52

In [6]:
"""
@author: Aayush Agrawal
@Purpose - Re-usable code in Python 3 for Recommender systems
ML-small-dataset - https://grouplens.org/datasets/movielens/
"""

import pandas as pd
import numpy as np
from scipy import sparse
from lightfm import LightFM
from sklearn.metrics.pairwise import cosine_similarity

def create_interaction_matrix(df,user_col, item_col, rating_col, norm= False, threshold = None):
    '''
    Function to create an interaction matrix dataframe from transactional type interactions
    Required Input -
        - df = Pandas DataFrame containing user-item interactions
        - user_col = column name containing user's identifier
        - item_col = column name containing item's identifier
        - rating col = column name containing user feedback on interaction with a given item
        - norm (optional) = True if a normalization of ratings is needed
        - threshold (required if norm = True) = value above which the rating is favorable
    Expected output - 
        - Pandas dataframe with user-item interactions ready to be fed in a recommendation algorithm
    '''
    interactions = df.groupby([user_col, item_col])[rating_col] \
            .sum().unstack().reset_index(). \
            fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

def create_user_dict(interactions):
    '''
    Function to create a user dictionary based on their index and number in interaction dataset
    Required Input - 
        interactions - dataset create by create_interaction_matrix
    Expected Output -
        user_dict - Dictionary type output containing interaction_index as key and user_id as value
    '''
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict
    
def create_item_dict(df,id_col,name_col):
    '''
    Function to create an item dictionary based on their item_id and item name
    Required Input - 
        - df = Pandas dataframe with Item information
        - id_col = Column name containing unique identifier for an item
        - name_col = Column name containing name of the item
    Expected Output -
        item_dict = Dictionary type output containing item_id as key and item_name as value
    '''
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

def runMF(interactions, n_components=30, loss='warp', k=15, epoch=30,n_jobs = 4):
    '''
    Function to run matrix-factorization algorithm
    Required Input -
        - interactions = dataset create by create_interaction_matrix
        - n_components = number of embeddings you want to create to define Item and user
        - loss = loss function other options are logistic, brp
        - epoch = number of epochs to run 
        - n_jobs = number of cores used for execution 
    Expected Output  -
        Model - Trained model
    '''
    x = sparse.csr_matrix(interactions.values)
    model = LightFM(no_components= n_components, loss=loss,k=k)
    model.fit(x,epochs=epoch,num_threads = n_jobs)
    return model

def sample_recommendation_user(model, interactions, user_id, user_dict, 
                               item_dict,threshold = 0,nrec_items = 10, show = True):
    '''
    Function to produce user recommendations
    Required Input - 
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - user_id = user ID for which we need to generate recommendation
        - user_dict = Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - threshold = value above which the rating is favorable in new interaction matrix
        - nrec_items = Number of output recommendation needed
    Expected Output - 
        - Prints list of items the given user has already bought
        - Prints list of N recommended items  which user hopefully will be interested in
    '''
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:] \
                                 [interactions.loc[user_id,:] > threshold].index) \
								 .sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    if show == True:
        print("Known Likes:")
        counter = 1
        for i in known_items:
            print(str(counter) + '- ' + i)
            counter+=1

        print("\n Recommended Items:")
        counter = 1
        for i in scores:
            print(str(counter) + '- ' + i)
            counter+=1
    return return_score_list
    

def sample_recommendation_item(model,interactions,item_id,user_dict,item_dict,number_of_user):
    '''
    Funnction to produce a list of top N interested users for a given item
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
        - item_id = item ID for which we need to generate recommended users
        - user_dict =  Dictionary type input containing interaction_index as key and user_id as value
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - number_of_user = Number of users needed as an output
    Expected Output -
        - user_list = List of recommended users 
    '''
    n_users, n_items = interactions.shape
    x = np.array(interactions.columns)
    scores = pd.Series(model.predict(np.arange(n_users), np.repeat(x.searchsorted(item_id),n_users)))
    user_list = list(interactions.index[scores.sort_values(ascending=False).head(number_of_user).index])
    return user_list 


def create_item_emdedding_distance_matrix(model,interactions):
    '''
    Function to create item-item distance embedding matrix
    Required Input -
        - model = Trained matrix factorization model
        - interactions = dataset used for training the model
    Expected Output -
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
    '''
    df_item_norm_sparse = sparse.csr_matrix(model.item_embeddings)
    similarities = cosine_similarity(df_item_norm_sparse)
    item_emdedding_distance_matrix = pd.DataFrame(similarities)
    item_emdedding_distance_matrix.columns = interactions.columns
    item_emdedding_distance_matrix.index = interactions.columns
    return item_emdedding_distance_matrix

def item_item_recommendation(item_emdedding_distance_matrix, item_id, 
                             item_dict, n_items = 10, show = True):
    '''
    Function to create item-item recommendation
    Required Input - 
        - item_emdedding_distance_matrix = Pandas dataframe containing cosine distance matrix b/w items
        - item_id  = item ID for which we need to generate recommended items
        - item_dict = Dictionary type input containing item_id as key and item_name as value
        - n_items = Number of items needed as an output
    Expected Output -
        - recommended_items = List of recommended items
    '''
    recommended_items = list(pd.Series(item_emdedding_distance_matrix.loc[item_id,:]. \
                                  sort_values(ascending = False).head(n_items+1). \
                                  index[1:n_items+1]))
    if show == True:
        print("Item of interest :{0}".format(item_dict[item_id]))
        print("Item similar to the above item:")
        counter = 1
        for i in recommended_items:
            print(str(counter) + '- ' +  item_dict[i])
            counter+=1
    return recommended_items

In [7]:
interactions = create_interaction_matrix(df = rating_df,
                                         user_col = 'user',
                                         item_col = 'blague',
                                         rating_col = 'rating')
interactions.head()

blague  1    2     5     7     8     10   11    13   14    15    16    17    18    19    20   21   22   23   25   26    27    28    29    31    32   34    35    36   37   38    39   40   41   42    43    44   45    46   47    48    49    50   51   52    53    54   55    56   57   58   59    60    61    62   63   64    65    66    67   68    69    70   71   72   73   74   75    76    77   78   79   80   81   82   83   84   85   86   87   88   89   90   91   92    93    95   96   97   98   99   100
user                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
0       0.0  0.0  0.00 -0.78  6.89  0.00  0.0  0.00  0.0 -1.31 -0.19  0.00  2.96 -0.29  1.17  0.0  0.0  0.0  0.0  0.0  0.00  0.00 -2.23  0.15  0.00  0.0  0.00  1.26  0.0  0.0  0.00  0.0  0.0  0.0 -7.52 -5.87  0.0  0.00  0.0 -8.20  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.00  0.0  0.0  0.00  0.00  2.23  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
1       0.0  0.0 -9.27 -9.17  0.00  0.00  0.0 -8.59  0.0 -8.59  0.00 -8.59  0.00 -6.41 -4.71  0.0  0.0  0.0  0.0  0.0 -4.85 -7.82  0.00 -9.56 -1.55  0.0 -9.47 -5.34  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0 -9.71  0.0 -9.71 -7.67 -3.59  0.0  0.0 -6.36 -9.66  0.0  0.00  0.0  0.0  0.0  0.00  0.00 -3.45  0.0  0.0 -7.18 -7.67  0.00  0.0 -8.54  0.00  0.0  0.0  0.0  0.0  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 -2.77  0.00  0.0  0.0  0.0  0.0  0.0
2       0.0  0.0 -6.12  0.00  0.00  0.00  0.0  0.00  0.0 -6.12  0.00 -1.99  0.00 -7.82  0.00  0.0  0.0  0.0  0.0  0.0  0.00  0.00  0.00  0.00  0.00  0.0  0.00 -6.65  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00  0.0  0.00  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0 -7.72  0.00  0.00  0.0  0.0  0.00  0.00  0.00  0.0  0.00 -7.67  0.0  0.0  0.0  0.0  0.0 -7.72  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0
3       0.0  0.0  0.00 -4.85  0.00  0.00  0.0  2.77  0.0 -3.69  0.00  0.00  3.40  0.00  0.00  0.0  0.0  0.0  0.0  0.0  3.01  0.00 -0.83  0.00 -5.15  0.0 -2.77 -1.80  0.0  0.0  0.00  0.0  0.0  0.0  0.00  0.00  0.0  0.00  0.0  0.00  0.00  0.00  0.0  0.0 -3.30  0.00  0.0  2.67  0.0  0.0  0.0  0.00  0.00  0.00  0.0  0.0  0.00  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.00  2.86  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  1.84  0.0  0.0  0.0  0.0  0.0
4       0.0  0.0  0.00  0.00  2.72 -8.59  0.0  0.00  0.0  6.89 -7.67  0.00  0.00  0.00  0.00  0.0  0.0  0.0  0.0  0.0 -7.48  0.00  8.93  0.00  0.00  0.0  0.00 -5.73  0.0  0.0  8.83  0.0  0.0  0.0  0.00  0.00  0.0  6.21  0.0  0.00  0.00  0.00  0.0  0.0  0.00  0.00  0.0  0.00  0.0  0.0  0.0  0.00  3.16  0.00  0.0  0.0  0.00  0.00  0.00  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.00  0.00  0.0  0.0  0.0  0.0  0.0

In [8]:
# Create User Dict
user_dict = create_user_dict(interactions=interactions)
# Create Item dict
movies_dict = create_item_dict(df = my_jokes_df,
                               id_col = 'id',
                               name_col = 'text')

In [9]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [15]:
import json
import math
n=len(set(rating_df["blague"]))  # N pour la formule
myprediction=[]
for i,j in rating_df.iterrows():
    #print(user_dict)
    #    print(movies_dict[j[1]]) n_users, n_items = interactions.shape
    if( ( j[0] in user_dict.keys() ) and ( j[1] in movies_dict.keys() ) ):
        try:
            scores = mf_model.predict(j[0],[j[1]])
        except:
            score=0
    else:
        scores=0
    #print(scores,j[2])
    myprediction.append(scores)
    #n_users, n_items = interactions.shape
    #user_x = user_dict[user_id]
    #scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    #myprediction.append(mf_model.predict(user_dict[int(j[0])],int(movies_dict[int(j[1])])))
    #print("note initiale",j[2],"prediction",note_moy)
rating_df["predictionNote"]=myprediction

In [ ]:
import json
import math
n=len(set(rating_df["blague"]))  # N pour la formule
myprediction=[]
for i,j in rating_df.iterrows():
    #if(j[2]==99):
    #print(j[0])
    #blague=my_jokes_df[ ( my_jokes_df["user"]==j[0] ) & ( my_jokes_df["blague"]==int(j[1]) ) ].text
    tmpline=my_jokes_df[ ( my_jokes_df["id"]==j[1])      ]
    for k,l in tmpline.iterrows():
        blague=json.loads(l[1])

    #print("blah",len(blague))
    #notesUtils=jokes_info_df[jokes_info_df.user==j[0]]#note donné par le meme utilisateur

    predict=0
    i=0
    for mot in blague:
        dd=jokes_info_df[jokes_info_df["mot"]==mot]["dfm"].values
        if(len(dd)==0):
            continue

        #print(dd)
        dfmmot=dd[0]
        #print(mot+str(dfmmot))


        note=jokes_info_df[
            ( jokes_info_df["user"] == j[0] )
            &
            ( jokes_info_df["mot"]==mot   )
        ].moyenne_user.values

        if(len(note)>0):
            #print(note[0])
            #predict +=   math.log(n/dfmmot)  *  note[0]# on normalise avec +1 car sinon risquons d'avoir un 0
            predict +=   note[0]
            i+=1
        #print("note",note)
    #print("izan",predict)
    if(i!=0):
        note_moy=predict/i
    else:
        note_moy=0
    #print("je vais mettre a jour",note_moy)

    myprediction.append(note_moy)
    #print("note initiale",j[2],"prediction",note_moy)
rating_df["predictionNote"]=myprediction

In [ ]:
moteur_recommandation_lightfm_bonus_df = rating_df # Compute a Pandas dataframe to write into model_recommandation



# Write recipe outputs
moteur_recommandation_lightfm_bonus = dataiku.Dataset("moteur_recommandation_lightfm_bonus")
moteur_recommandation_lightfm_bonus.write_with_schema(moteur_recommandation_lightfm_bonus_df)